<a href="https://colab.research.google.com/github/AndresPautrat/Concurrente/blob/main/go_CliServ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install golang-go 1>/dev/null && go version



go version go1.10.4 linux/amd64


In [ ]:
%%writefile serv.go
package main

import (
    "bufio"
    "fmt"
    "net"
)

func main(){
    ln,_ :=net.Listen("tcp","localhost:8000")
    defer ln.Close()
    con,_ := ln.Accept()

    defer con.Close()

    r := bufio.NewReader(con)
    msg, _ := r.ReadString('\n')
    fmt.Println(msg)
}


Overwriting serv.go


In [ ]:
%%writefile cli.go
package main

import (
    "fmt"
    "net"
)

func main(){
    con,_ := net.Dial("tcp","localhost:8000")
    defer con.Close()

    fmt.Fprintln(con, "Hola soy un cliente")
}


Overwriting cli.go


In [ ]:
%%script bash
go build serv.go
go build cli.go

./serv &
sleep 1
./cli

Hola soy un cliente



In [ ]:
%%writefile main.go
package main

import (
    "fmt"
)

func proc(inC <-chan float64, outC chan<- float64) {
    min := float64(1<<62)
    for num := range inC {
        if num < min {
            outC<- min
            min = num
        } else {
            outC<- num
        }
    }
    fmt.Printf("%.0f ", min)
    close(outC)
}

func main() {
    nums := []float64{ 6., 1., 8., 10., 5., 9., 3., 7., 2., 4. }
    ch := make([]chan float64, len(nums) + 1)
    ch[0] = make(chan float64)
    for i := range nums {
        ch[i+1] = make(chan float64)
        go proc(ch[i], ch[i+1])
    }
    go func(){
        for _, num := range nums {
            ch[0]<- num
        }
        close(ch[0])
    }()
    for range ch[len(nums)] {
    }
}

Writing main.go


In [ ]:
!go run main.go

1 2 3 4 5 6 7 8 9 10 

In [ ]:
%%writefile serv.go
package main

import (
    "bufio"
    "fmt"
    "net"
)

func main(){
    ln,_ :=net.Listen("tcp","localhost:8000")
    defer ln.Close()
    con,_ := ln.Accept()

    defer con.Close()

    r := bufio.NewReader(con)
    msg, _ := r.ReadString('\n')
    fmt.Println(msg)
}

In [ ]:
%%writefile cli.go
package main

import (
    "fmt"
    "net"
)

func main(){
    con,_ := net.Dial("tcp","localhost:8000")
    defer con.Close()

    fmt.Fprintln(con, "Hola soy un cliente")
}